In [3]:
from langchain_huggingface import ChatHuggingFace,HuggingFaceEndpoint
from dotenv import load_dotenv
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate,PromptTemplate
from pydantic import BaseModel,Field
from typing import Annotated,Optional
load_dotenv()

True

# chains types
            sequantial chains
            parallel chains
            conditional chains


# sequantial chains

In [4]:
model=HuggingFaceEndpoint(repo_id="deepseek-ai/DeepSeek-V3.2",
                        temperature= 0.7
                        ,task="text-generation")
model=ChatHuggingFace(llm=model)


d:\AI_agent_repo\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Each scene (image/video prompt) under a topic
class Scene(BaseModel):
    prompt: Annotated[str, Field(description="Describe the detailed scene for video generation")]

# Each topic contains title + list of scenes
class CarTopic(BaseModel):
    title: Annotated[str, Field(description="Title of the car topic")]
    scenes: Annotated[list[Scene], Field(description="List of scenes for this topic")]

# Main structure: 3 car topics
class YTCarTopics(BaseModel):
    topics: Annotated[list[CarTopic],Field(description="List of trending car topics")]


parser = PydanticOutputParser(pydantic_object=YTCarTopics)


prompt_template = PromptTemplate(
    template="""
search trending car topics. 
For each topic, provide:
- title
- 5 scenes (descriptive prompts for image/video generation)

Return the output in JSON following these instructions:

{format_instructions}
""",
    input_variables=[],
    partial_variables={
        "format_instructions": parser.get_format_instructions()
    }
)



In [6]:
chain = prompt_template | model | parser


In [7]:

print(chain.get_graph().print_ascii())

    +-------------+      
    | PromptInput |      
    +-------------+      
            *            
            *            
            *            
   +----------------+    
   | PromptTemplate |    
   +----------------+    
            *            
            *            
            *            
  +-----------------+    
  | ChatHuggingFace |    
  +-----------------+    
            *            
            *            
            *            
+----------------------+ 
| PydanticOutputParser | 
+----------------------+ 
            *            
            *            
            *            
    +-------------+      
    | YTCarTopics |      
    +-------------+      
None


In [8]:
import json

final_result = chain.invoke({})  # no input variables needed
# Convert to dict for pretty JSON
data_dict = final_result.model_dump()
print(json.dumps(data_dict, indent=2))

OutputParserException: Failed to parse YTCarTopics from completion {"topics": [{"title": "Electric Vehicle Road Trip Adventures", "scenes": [{"prompt": "A sleek electric SUV charging at a futuristic fast-charging station at sunset, with a mountain range in the background and a family stretching their legs nearby."}, {"prompt": "A panoramic shot from inside an EV on a scenic coastal highway, dashboard screen showing remaining range and nearby charging points, ocean waves crashing against cliffs."}, {"prompt": "An electric pickup truck towing a small camper trailer through a red rock desert landscape, solar panels on the camper roof glowing in the midday sun."}, {"prompt": "A family using their EV's vehicle-to-load capability to power a campsite at night, with string lights, a small electric grill, and laptops charging from the car."}, {"prompt": "An EV navigating through a dramatic mountain pass during autumn, leaves changing color, with the regenerative braking indicator prominently di

In [ ]:
with open("ytb_car_topics.json", "w", encoding="utf-8") as f:
    json.dump(data_dict, f, indent=2, ensure_ascii=False)

# parallel chain

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

images_script = PromptTemplate(
    template="""
You are a cinematic storyboard creator.

Create animated image scene prompts for a YouTube video.

Topic: "{topic}"
Total Video Duration: {seconds} seconds
Each scene duration: 6 seconds

For each scene include:
- Scene Title
- Detailed visual description
- Camera movement
- Lighting description
- Environment details
- Animation motion description

Make scenes dynamic, engaging, and highly detailed.

Return the output as structured text.
""",
    input_variables=["topic", "seconds"]
)

Audio_script = PromptTemplate(
    template="""
You are a professional YouTube script writer.

Write an engaging voice-over narration script for a video about:

Topic: "{topic}"
Total Duration: {seconds} seconds

Requirements:
- Strong hook in the first 5 seconds
- Clear and engaging storytelling
- Conversational but professional tone
- Short sentences for natural voice generation
- End with a strong closing statement or call to action
- Match pacing for a {seconds}-second video

Return only the narration script.
""",
    input_variables=["topic", "seconds"]
)

merge_script = PromptTemplate(
    template="""
You are a video production assistant.

Merge the image scenes and narration into a synchronized storyboard.

 Topic: "{topic}"
 Total Duration: {seconds} seconds

Instructions:
- Each scene lasts 6 seconds
- Align narration naturally with each scene
- Keep smooth transitions between scenes
- Ensure audio matches visual content

Image Scenes:
{image_scenes}

Audio Script:
{audio_script}

Return output in this format:

Scene 1 (0-6s):
Visual:
Narration:

Scene 2 (6-12s):
Visual:
Narration:

Continue until the full video duration is covered.
""",
    input_variables=["topic", "seconds", "image_scenes", "audio_script"]
)


In [ ]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()

parallel_chain = RunnableParallel(
    image_scenes=images_script | model | parser,
    audio_script=Audio_script | model | parser,
)



chain = (
    RunnableParallel(
        {
            "topic": RunnablePassthrough(),
            "seconds": RunnablePassthrough(),
            "image_scenes": parallel_chain,
            "audio_script": parallel_chain,
        })
    | merge_script
    | model
    | parser
)

In [ ]:

result = chain.invoke({
    "topic": "Electric Supercars 2026",
    "seconds": 30
})

print(result)

Scene 1 (0-6s):
Visual: A sleek, matte-black electric supercar rests in a minimalist, futuristic showroom. Its surface pulses with a soft, neon-blue underglow and cyan circuit-like patterns. Holographic diagnostic screens and spec sheets (showing 0-60: 1.8s) float in the air. A slow, arcing dolly shot circles the car in the cool dawn light.
Narration: The roar of engines is fading. Welcome to the silent revolution of 2026.

Scene 2 (6-12s):
Visual: The supercar accelerates through a rain-slicked, neon-drenched metropolitan highway at night. Skyscrapers with digital billboards tower above. The car's tires kick up mist, and its brake calipers glow cyan/magenta. A dynamic low-angle chase shot tracks the vehicle.
Narration: These aren’t just cars; they’re physics-defying masterpieces.

Scene 3 (12-18s):
Visual: The supercar attacks a winding cliffside road at sunrise. (Based on the audio script's Scene 3 description. A detailed visual would show the car navigating sharp turns against a maj

In [ ]:
print(chain.get_graph().print_ascii())

                                                                        +--------------------------------------------------------+                                                                 
                                                                        | Parallel<topic,seconds,image_scenes,audio_script>Input |                                                                 
                                                                      **+--------------------------------------------------------+*******                                                          
                                                          ************                            *                       *********      **************                                            
                                              ************                                       *                                 *********           **************                              
                    

# conditional chain

In [22]:
from langchain_core.runnables import RunnableBranch,RunnableLambda
from pydantic import BaseModel, Field
from typing import Literal
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

In [25]:
class Feedback(BaseModel):

    sentiment: Literal['positive', 'negative'] = Field(description='Give the sentiment of the feedback')

parser2 = PydanticOutputParser(pydantic_object=Feedback)

prompt1 = PromptTemplate(
    template='Classify the sentiment of the following feedback text into postive or negative \n {feedback} \n {format_instruction}',
    input_variables=['feedback'],
    partial_variables={'format_instruction':parser2.get_format_instructions()}
)

classifier_chain = prompt1 | model | parser2

prompt2 = PromptTemplate(
    template='Write an appropriate response to this positive feedback \n {feedback}',
    input_variables=['feedback']
)

prompt3 = PromptTemplate(
    template='Write an appropriate response to this negative feedback \n {feedback}',
    input_variables=['feedback']
)

branch_chain = RunnableBranch(
    (lambda x:x.sentiment == 'positive', prompt2 | model | parser),
    (lambda x:x.sentiment == 'negative', prompt3 | model | parser),
    RunnableLambda(lambda x: "could not find sentiment")
)

chain = classifier_chain | branch_chain

print(chain.invoke({'feedback': 'This is a bad phone not good style'}))



Thank you for sharing your feedback. I'm sorry to hear that your experience didn't meet your expectations, and I appreciate you taking the time to let us know. Your comments are important to us, and we'd like to better understand what happened so we can address your concerns. Please feel free to share more details, and we’ll do our best to resolve the issue and improve moving forward.


In [18]:
chain.get_graph().print_ascii()

    +-------------+      
    | PromptInput |      
    +-------------+      
            *            
            *            
            *            
   +----------------+    
   | PromptTemplate |    
   +----------------+    
            *            
            *            
            *            
  +-----------------+    
  | ChatHuggingFace |    
  +-----------------+    
            *            
            *            
            *            
+----------------------+ 
| PydanticOutputParser | 
+----------------------+ 
            *            
            *            
            *            
       +--------+        
       | Branch |        
       +--------+        
            *            
            *            
            *            
    +--------------+     
    | BranchOutput |     
    +--------------+     
